In [1]:
import os

In [2]:
%pwd

'd:\\git\\Chest-Disease-Classification-from-Chest-CT-Scan-Image\\research'

In [3]:
%pwd

'd:\\git\\Chest-Disease-Classification-from-Chest-CT-Scan-Image\\research'

In [4]:
os.chdir('d:\\git\\Chest-Disease-Classification-from-Chest-CT-Scan-Image\\')

In [5]:
%pwd

'd:\\git\\Chest-Disease-Classification-from-Chest-CT-Scan-Image'

In [6]:
os.environ["MLFLOW_TRACKING_URI"]="https://dagshub.com/mayankchugh-learning/Chest-Disease-Classification-from-Chest-CT-Scan-Image.mlflow"
os.environ["MLFLOW_TRACKING_USERNAME"]="mayankchugh-learning"
os.environ["MLFLOW_TRACKING_PASSWORD"]=""

In [15]:
%pwd

'd:\\git\\Chest-Disease-Classification-from-Chest-CT-Scan-Image'

In [21]:
!python main.py


[2024-03-15 12:57:57,709: INFO: main: >>>>>> stage Data Ingestion stage started <<<<<<]
[2024-03-15 12:57:57,721: INFO: common: yaml file: config\config.yaml loaded successfully]
[2024-03-15 12:57:57,734: INFO: common: yaml file: params.yaml loaded successfully]
[2024-03-15 12:57:57,735: INFO: common: created directory at: artifacts]
[2024-03-15 12:57:57,736: INFO: common: created directory at: artifacts/data_ingestion]
[2024-03-15 12:57:57,737: INFO: data_ingestion: Downloading data from https://drive.google.com/file/d/1RE-2BwQGjQdUao9F8pvGEbZe2wZHFQVQ/view?usp=sharing into file artifacts/data_ingestion/data.zip]
[2024-03-15 12:58:08,664: INFO: data_ingestion: Downloaded data from https://drive.google.com/file/d/1RE-2BwQGjQdUao9F8pvGEbZe2wZHFQVQ/view?usp=sharing into file artifacts/data_ingestion/data.zip]
[2024-03-15 12:58:10,291: INFO: main: >>>>>> stage Data Ingestion stage completed <<<<<<

x==========x]
[2024-03-15 12:58:10,291: INFO: main: *******************]
[2024-03-15 12:58:

Downloading...
From (original): https://drive.google.com/uc?/export=download&id=1RE-2BwQGjQdUao9F8pvGEbZe2wZHFQVQ
From (redirected): https://drive.google.com/uc?%2Fexport=download&id=1RE-2BwQGjQdUao9F8pvGEbZe2wZHFQVQ&confirm=t&uuid=4071b8f9-9aa8-48c7-bf33-b20d5d85c832
To: d:\git\Chest-Disease-Classification-from-Chest-CT-Scan-Image\artifacts\data_ingestion\data.zip

  0%|          | 0.00/49.0M [00:00<?, ?B/s]
  2%|▏         | 1.05M/49.0M [00:00<00:06, 7.44MB/s]
  4%|▍         | 2.10M/49.0M [00:00<00:06, 7.40MB/s]
  6%|▋         | 3.15M/49.0M [00:00<00:06, 7.38MB/s]
  9%|▊         | 4.19M/49.0M [00:00<00:06, 7.12MB/s]
 11%|█         | 5.24M/49.0M [00:00<00:06, 6.96MB/s]
 13%|█▎        | 6.29M/49.0M [00:00<00:06, 6.97MB/s]
 15%|█▍        | 7.34M/49.0M [00:01<00:06, 6.89MB/s]
 17%|█▋        | 8.39M/49.0M [00:01<00:05, 7.25MB/s]
 19%|█▉        | 9.44M/49.0M [00:01<00:05, 7.45MB/s]
 21%|██▏       | 10.5M/49.0M [00:01<00:05, 7.42MB/s]
 24%|██▎       | 11.5M/49.0M [00:01<00:05, 7.32MB/s]
 26%

## Entity Code

In [7]:
import tensorflow as tf

In [8]:
model = tf.keras.models.load_model("artifacts/training/model.h5")

In [9]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class EvaluationConfig:
    path_of_model: Path
    training_data: Path
    all_params: dict
    mlflow_uri: str
    params_image_size: list
    params_batch_size: int

## ConfigurationManager file code

In [10]:
from cnnClassifier.constants import *
from cnnClassifier.utils.common import read_yaml, create_directories, save_json

In [11]:
class ConfigurationManager:
    def __init__(
        self, 
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_directories([self.config.artifacts_root])


    def get_evaluation_config(self) -> EvaluationConfig:
        eval_config = EvaluationConfig(
            path_of_model="artifacts/training/model.h5",
            training_data="artifacts/data_ingestion/Chest-CT-Scan-data",
            mlflow_uri="https://dagshub.com/mayankchugh-learning/Chest-Disease-Classification-from-Chest-CT-Scan-Image.mlflow",
            all_params=self.params,
            params_image_size=self.params.IMAGE_SIZE,
            params_batch_size=self.params.BATCH_SIZE
        )
        return eval_config




## Model Evaluation component file code

In [12]:
import tensorflow as tf
from pathlib import Path
import mlflow
import mlflow.keras
from urllib.parse import urlparse

In [13]:
class Evaluation:
    def __init__(self, config: EvaluationConfig):
        self.config = config

    
    def _valid_generator(self):

        datagenerator_kwargs = dict(
            rescale = 1./255,
            validation_split=0.30
        )

        dataflow_kwargs = dict(
            target_size=self.config.params_image_size[:-1],
            batch_size=self.config.params_batch_size,
            interpolation="bilinear"
        )

        valid_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
            **datagenerator_kwargs
        )

        self.valid_generator = valid_datagenerator.flow_from_directory(
            directory=self.config.training_data,
            subset="validation",
            shuffle=False,
            **dataflow_kwargs
        )


    @staticmethod
    def load_model(path: Path) -> tf.keras.Model:
        return tf.keras.models.load_model(path)
    

    def evaluation(self):
        self.model = self.load_model(self.config.path_of_model)
        self._valid_generator()
        self.score = model.evaluate(self.valid_generator)
        self.save_score()

    def save_score(self):
        scores = {"loss": self.score[0], "accuracy": self.score[1]}
        save_json(path=Path("scores.json"), data=scores)

    
    def log_into_mlflow(self):
        mlflow.set_registry_uri(self.config.mlflow_uri)
        tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme
        
        with mlflow.start_run():
            mlflow.log_params(self.config.all_params)
            mlflow.log_metrics(
                {"loss": self.score[0], "accuracy": self.score[1]}
            )
            # Model registry does not work with file store
            if tracking_url_type_store != "file":

                # Register the model
                # There are other ways to use the Model Registry, which depends on the use case,
                # please refer to the doc for more information:
                # https://mlflow.org/docs/latest/model-registry.html#api-workflow
                mlflow.keras.log_model(self.model, "model", registered_model_name="VGG16Model")
            else:
                mlflow.keras.log_model(self.model, "model")


## stage 04 Model Evalution pipeline file code

In [14]:
try:
    config = ConfigurationManager()
    eval_config = config.get_evaluation_config()
    evaluation = Evaluation(eval_config)
    evaluation.evaluation()
    evaluation.log_into_mlflow()

except Exception as e:
   raise e

[2024-03-15 11:28:15,103: INFO: common: yaml file: config\config.yaml loaded successfully]
[2024-03-15 11:28:15,112: INFO: common: yaml file: params.yaml loaded successfully]
[2024-03-15 11:28:15,115: INFO: common: created directory at: artifacts]
Found 102 images belonging to 2 classes.
7/7 [==============================] - 79s 11s/step - loss: 6.5524 - accuracy: 0.4314
[2024-03-15 11:29:37,248: INFO: common: json file saved at: scores.json]


2024/03/15 11:29:39 WARNING mlflow.utils.git_utils: Failed to import Git (the Git executable is probably not on your PATH), so Git SHA is not available. Error: Failed to initialize: Bad git executable.
The git executable must be specified in one of the following ways:
    - be included in your $PATH
    - be set via $GIT_PYTHON_GIT_EXECUTABLE
    - explicitly set via git.refresh()

All git commands will error until this is rectified.

This initial message can be silenced or aggravated in the future by setting the
$GIT_PYTHON_REFRESH environment variable. Use one of the following values:
    - quiet|q|silence|s|silent|none|n|0: for no message or exception
    - warn|w|warning|log|l|1: for a warning message (logged at level CRITICAL, displayed by default)
    - error|e|exception|raise|r|2: for a raised exception

Example:
    export GIT_PYTHON_REFRESH=quiet

2024/03/15 11:29:44 WARNING mlflow.tensorflow: You are saving a TensorFlow Core model or Keras model without a signature. Inference

[2024-03-15 11:29:54,319: WARNING: save: Found untraced functions such as _jit_compiled_convolution_op, _jit_compiled_convolution_op, _jit_compiled_convolution_op, _jit_compiled_convolution_op, _jit_compiled_convolution_op while saving (showing 5 of 14). These functions will not be directly callable after loading.]
INFO:tensorflow:Assets written to: C:\Users\mayan\AppData\Local\Temp\tmph3ikgioy\model\data\model\assets
[2024-03-15 11:29:59,419: INFO: builder_impl: Assets written to: C:\Users\mayan\AppData\Local\Temp\tmph3ikgioy\model\data\model\assets]


c:\Users\mayan\.conda\envs\chest\lib\site-packages\_distutils_hack\__init__.py:33: UserWarning: Setuptools is replacing distutils.
  warnings.warn("Setuptools is replacing distutils.")
Successfully registered model 'VGG16Model'.
2024/03/15 11:31:07 INFO mlflow.tracking._model_registry.client: Waiting up to 300 seconds for model version to finish creation.                     Model name: VGG16Model, version 1
Created version '1' of model 'VGG16Model'.


In [22]:
!python main.py

[2024-03-15 13:07:54,235: INFO: main: >>>>>> stage Data Ingestion stage started <<<<<<]
[2024-03-15 13:07:54,238: INFO: common: yaml file: config\config.yaml loaded successfully]
[2024-03-15 13:07:54,244: INFO: common: yaml file: params.yaml loaded successfully]
[2024-03-15 13:07:54,249: INFO: common: created directory at: artifacts]
[2024-03-15 13:07:54,250: INFO: common: created directory at: artifacts/data_ingestion]
[2024-03-15 13:07:54,251: INFO: data_ingestion: Downloading data from https://drive.google.com/file/d/1RE-2BwQGjQdUao9F8pvGEbZe2wZHFQVQ/view?usp=sharing into file artifacts/data_ingestion/data.zip]
[2024-03-15 13:08:04,049: INFO: data_ingestion: Downloaded data from https://drive.google.com/file/d/1RE-2BwQGjQdUao9F8pvGEbZe2wZHFQVQ/view?usp=sharing into file artifacts/data_ingestion/data.zip]
[2024-03-15 13:08:05,829: INFO: main: >>>>>> stage Data Ingestion stage completed <<<<<<

x==========x]
[2024-03-15 13:08:05,830: INFO: main: *******************]
[2024-03-15 13:08:

Downloading...
From (original): https://drive.google.com/uc?/export=download&id=1RE-2BwQGjQdUao9F8pvGEbZe2wZHFQVQ
From (redirected): https://drive.google.com/uc?%2Fexport=download&id=1RE-2BwQGjQdUao9F8pvGEbZe2wZHFQVQ&confirm=t&uuid=112a6bcd-44fc-4d6e-b1e2-9eb79d5bf4fd
To: d:\git\Chest-Disease-Classification-from-Chest-CT-Scan-Image\artifacts\data_ingestion\data.zip

  0%|          | 0.00/49.0M [00:00<?, ?B/s]
  2%|▏         | 1.05M/49.0M [00:00<00:07, 6.84MB/s]
  4%|▍         | 2.10M/49.0M [00:00<00:06, 7.21MB/s]
  6%|▋         | 3.15M/49.0M [00:00<00:06, 6.77MB/s]
  9%|▊         | 4.19M/49.0M [00:00<00:06, 6.99MB/s]
 11%|█         | 5.24M/49.0M [00:00<00:05, 7.42MB/s]
 13%|█▎        | 6.29M/49.0M [00:00<00:05, 7.42MB/s]
 15%|█▍        | 7.34M/49.0M [00:01<00:05, 7.11MB/s]
 17%|█▋        | 8.39M/49.0M [00:01<00:05, 7.32MB/s]
 19%|█▉        | 9.44M/49.0M [00:01<00:05, 7.44MB/s]
 21%|██▏       | 10.5M/49.0M [00:01<00:04, 7.74MB/s]
 24%|██▎       | 11.5M/49.0M [00:01<00:04, 7.92MB/s]
 26%

In [24]:
!python main.py

[2024-03-15 13:18:12,039: INFO: main: >>>>>> stage Data Ingestion stage started <<<<<<]
[2024-03-15 13:18:12,043: INFO: common: yaml file: config\config.yaml loaded successfully]
[2024-03-15 13:18:12,046: INFO: common: yaml file: params.yaml loaded successfully]
[2024-03-15 13:18:12,046: INFO: common: created directory at: artifacts]
[2024-03-15 13:18:12,048: INFO: common: created directory at: artifacts/data_ingestion]
[2024-03-15 13:18:12,050: INFO: data_ingestion: Downloading data from https://drive.google.com/file/d/1RE-2BwQGjQdUao9F8pvGEbZe2wZHFQVQ/view?usp=sharing into file artifacts/data_ingestion/data.zip]
[2024-03-15 13:18:23,609: INFO: data_ingestion: Downloaded data from https://drive.google.com/file/d/1RE-2BwQGjQdUao9F8pvGEbZe2wZHFQVQ/view?usp=sharing into file artifacts/data_ingestion/data.zip]
[2024-03-15 13:18:24,761: INFO: main: >>>>>> stage Data Ingestion stage completed <<<<<<

x==========x]
[2024-03-15 13:18:24,761: INFO: main: *******************]
[2024-03-15 13:18:

Downloading...
From (original): https://drive.google.com/uc?/export=download&id=1RE-2BwQGjQdUao9F8pvGEbZe2wZHFQVQ
From (redirected): https://drive.google.com/uc?%2Fexport=download&id=1RE-2BwQGjQdUao9F8pvGEbZe2wZHFQVQ&confirm=t&uuid=09da8487-eb99-4bef-a1d4-16c7dc3cf7d2
To: d:\git\Chest-Disease-Classification-from-Chest-CT-Scan-Image\artifacts\data_ingestion\data.zip

  0%|          | 0.00/49.0M [00:00<?, ?B/s]
  1%|          | 524k/49.0M [00:00<00:11, 4.26MB/s]
  2%|▏         | 1.05M/49.0M [00:00<00:10, 4.50MB/s]
  4%|▍         | 2.10M/49.0M [00:00<00:08, 5.60MB/s]
  6%|▋         | 3.15M/49.0M [00:01<00:19, 2.33MB/s]
  7%|▋         | 3.67M/49.0M [00:01<00:17, 2.53MB/s]
  9%|▊         | 4.19M/49.0M [00:01<00:15, 2.83MB/s]
 10%|▉         | 4.72M/49.0M [00:01<00:14, 3.10MB/s]
 11%|█         | 5.24M/49.0M [00:01<00:13, 3.32MB/s]
 12%|█▏        | 5.77M/49.0M [00:01<00:12, 3.51MB/s]
 13%|█▎        | 6.29M/49.0M [00:01<00:11, 3.68MB/s]
 14%|█▍        | 6.82M/49.0M [00:02<00:11, 3.79MB/s]
 15%|

In [25]:
!python main.py

[2024-03-15 14:39:02,570: INFO: main: >>>>>> stage Data Ingestion stage started <<<<<<]
[2024-03-15 14:39:02,575: INFO: common: yaml file: config\config.yaml loaded successfully]
[2024-03-15 14:39:02,577: INFO: common: yaml file: params.yaml loaded successfully]
[2024-03-15 14:39:02,578: INFO: common: created directory at: artifacts]
[2024-03-15 14:39:02,578: INFO: common: created directory at: artifacts/data_ingestion]
[2024-03-15 14:39:02,580: INFO: data_ingestion: Downloading data from https://drive.google.com/file/d/1RE-2BwQGjQdUao9F8pvGEbZe2wZHFQVQ/view?usp=sharing into file artifacts/data_ingestion/data.zip]
[2024-03-15 14:39:13,220: INFO: data_ingestion: Downloaded data from https://drive.google.com/file/d/1RE-2BwQGjQdUao9F8pvGEbZe2wZHFQVQ/view?usp=sharing into file artifacts/data_ingestion/data.zip]
[2024-03-15 14:39:14,474: INFO: main: >>>>>> stage Data Ingestion stage completed <<<<<<

x==========x]
[2024-03-15 14:39:14,475: INFO: main: *******************]
[2024-03-15 14:39:

Downloading...
From (original): https://drive.google.com/uc?/export=download&id=1RE-2BwQGjQdUao9F8pvGEbZe2wZHFQVQ
From (redirected): https://drive.google.com/uc?%2Fexport=download&id=1RE-2BwQGjQdUao9F8pvGEbZe2wZHFQVQ&confirm=t&uuid=a55074f2-e439-4a70-a839-d216094f7593
To: d:\git\Chest-Disease-Classification-from-Chest-CT-Scan-Image\artifacts\data_ingestion\data.zip

  0%|          | 0.00/49.0M [00:00<?, ?B/s]
  1%|          | 524k/49.0M [00:00<00:11, 4.37MB/s]
  3%|▎         | 1.57M/49.0M [00:00<00:08, 5.75MB/s]
  5%|▌         | 2.62M/49.0M [00:00<00:07, 6.15MB/s]
  7%|▋         | 3.67M/49.0M [00:00<00:06, 6.69MB/s]
 10%|▉         | 4.72M/49.0M [00:00<00:08, 5.53MB/s]
 12%|█▏        | 5.77M/49.0M [00:01<00:07, 5.44MB/s]
 14%|█▍        | 6.82M/49.0M [00:01<00:07, 5.52MB/s]
 16%|█▌        | 7.86M/49.0M [00:01<00:07, 5.74MB/s]
 18%|█▊        | 8.91M/49.0M [00:01<00:06, 6.19MB/s]
 20%|██        | 9.96M/49.0M [00:01<00:06, 6.29MB/s]
 22%|██▏       | 11.0M/49.0M [00:01<00:05, 6.76MB/s]
 25%|

In [26]:
!python main.py

[2024-03-15 14:47:46,816: INFO: main: >>>>>> stage Data Ingestion stage started <<<<<<]

Downloading...
From (original): https://drive.google.com/uc?/export=download&id=1RE-2BwQGjQdUao9F8pvGEbZe2wZHFQVQ
From (redirected): https://drive.google.com/uc?%2Fexport=download&id=1RE-2BwQGjQdUao9F8pvGEbZe2wZHFQVQ&confirm=t&uuid=05390b04-f5bf-47e6-a4f9-63a1eadd04ef
To: d:\git\Chest-Disease-Classification-from-Chest-CT-Scan-Image\artifacts\data_ingestion\data.zip

  0%|          | 0.00/49.0M [00:00<?, ?B/s]
  2%|▏         | 1.05M/49.0M [00:00<00:06, 6.88MB/s]
  4%|▍         | 2.10M/49.0M [00:00<00:05, 8.27MB/s]
  6%|▋         | 3.15M/49.0M [00:00<00:05, 8.22MB/s]
  9%|▊         | 4.19M/49.0M [00:00<00:05, 8.11MB/s]
 11%|█         | 5.24M/49.0M [00:00<00:05, 8.00MB/s]
 13%|█▎        | 6.29M/49.0M [00:00<00:05, 7.46MB/s]
 15%|█▍        | 7.34M/49.0M [00:00<00:05, 7.91MB/s]
 17%|█▋        | 8.39M/49.0M [00:01<00:05, 7.35MB/s]
 19%|█▉        | 9.44M/49.0M [00:01<00:05, 6.87MB/s]
 21%|██▏       | 10.5M/49.0M [00:01<00:05, 6.75MB/s]
 24%|██▎       | 11.5M/49.0M [00:01<00:05, 7.09MB/s]
 26%


[2024-03-15 14:47:46,818: INFO: common: yaml file: config\config.yaml loaded successfully]
[2024-03-15 14:47:46,826: INFO: common: yaml file: params.yaml loaded successfully]
[2024-03-15 14:47:46,830: INFO: common: created directory at: artifacts]
[2024-03-15 14:47:46,833: INFO: common: created directory at: artifacts/data_ingestion]
[2024-03-15 14:47:46,836: INFO: data_ingestion: Downloading data from https://drive.google.com/file/d/1RE-2BwQGjQdUao9F8pvGEbZe2wZHFQVQ/view?usp=sharing into file artifacts/data_ingestion/data.zip]
[2024-03-15 14:47:57,355: INFO: data_ingestion: Downloaded data from https://drive.google.com/file/d/1RE-2BwQGjQdUao9F8pvGEbZe2wZHFQVQ/view?usp=sharing into file artifacts/data_ingestion/data.zip]
[2024-03-15 14:47:59,194: INFO: main: >>>>>> stage Data Ingestion stage completed <<<<<<

x==========x]
[2024-03-15 14:47:59,194: INFO: main: *******************]
[2024-03-15 14:47:59,194: INFO: main: >>>>>> stage Prepare base model started <<<<<<]
[2024-03-15 14:47:5

In [27]:
!python main.py

[2024-03-15 15:52:46,378: INFO: main: >>>>>> stage Data Ingestion stage started <<<<<<]
[2024-03-15 15:52:46,397: INFO: common: yaml file: config\config.yaml loaded successfully]
[2024-03-15 15:52:46,400: INFO: common: yaml file: params.yaml loaded successfully]
[2024-03-15 15:52:46,401: INFO: common: created directory at: artifacts]
[2024-03-15 15:52:46,401: INFO: common: created directory at: artifacts/data_ingestion]
[2024-03-15 15:52:46,402: INFO: data_ingestion: Downloading data from https://drive.google.com/file/d/1RE-2BwQGjQdUao9F8pvGEbZe2wZHFQVQ/view?usp=sharing into file artifacts/data_ingestion/data.zip]
[2024-03-15 15:52:57,654: INFO: data_ingestion: Downloaded data from https://drive.google.com/file/d/1RE-2BwQGjQdUao9F8pvGEbZe2wZHFQVQ/view?usp=sharing into file artifacts/data_ingestion/data.zip]
[2024-03-15 15:53:00,387: INFO: main: >>>>>> stage Data Ingestion stage completed <<<<<<

x==========x]
[2024-03-15 15:53:00,387: INFO: main: *******************]
[2024-03-15 15:53:

Downloading...
From (original): https://drive.google.com/uc?/export=download&id=1RE-2BwQGjQdUao9F8pvGEbZe2wZHFQVQ
From (redirected): https://drive.google.com/uc?%2Fexport=download&id=1RE-2BwQGjQdUao9F8pvGEbZe2wZHFQVQ&confirm=t&uuid=d12ced37-a6c2-47c8-b6e7-c66026e4fe78
To: d:\git\Chest-Disease-Classification-from-Chest-CT-Scan-Image\artifacts\data_ingestion\data.zip

  0%|          | 0.00/49.0M [00:00<?, ?B/s]
  2%|▏         | 1.05M/49.0M [00:00<00:06, 6.93MB/s]
  4%|▍         | 2.10M/49.0M [00:00<00:06, 7.48MB/s]
  6%|▋         | 3.15M/49.0M [00:00<00:05, 7.98MB/s]
  9%|▊         | 4.19M/49.0M [00:00<00:05, 7.95MB/s]
 11%|█         | 5.24M/49.0M [00:00<00:05, 8.36MB/s]
 13%|█▎        | 6.29M/49.0M [00:00<00:04, 8.65MB/s]
 15%|█▍        | 7.34M/49.0M [00:00<00:04, 8.79MB/s]
 17%|█▋        | 8.39M/49.0M [00:01<00:04, 8.76MB/s]
 19%|█▉        | 9.44M/49.0M [00:01<00:04, 8.94MB/s]
 21%|██▏       | 10.5M/49.0M [00:01<00:04, 8.90MB/s]
 24%|██▎       | 11.5M/49.0M [00:01<00:04, 8.56MB/s]
 26%